Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

## Build Spark Session

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession
.builder
.appName("Titanic_Prediction")
.getOrCreate())

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [3]:
df_train = spark.read.csv('train.csv',header=True,inferSchema=True)
df_test = spark.read.csv('test.csv',header=True,inferSchema=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [4]:
type(df_train)

pyspark.sql.dataframe.DataFrame

**Show 5 rows.**

In [5]:
df_train.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [6]:
df_train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [7]:
df_train.describe().show(truncate=True)

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [8]:
df_train.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [9]:
df_train.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [10]:
df_survived = df_train.groupBy('Survived').count()

**Display your result:**

In [11]:
df_survived.show() # 1 for survived

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



**Can you display your answer in ratio form?(Hint: Use UDF.)**






In [12]:
from pyspark.sql.functions import *

In [13]:
df_survived.select('Survived',(col("count")/df_train.count()).alias('Survived Ratio')).show()

+--------+------------------+
|Survived|    Survived Ratio|
+--------+------------------+
|       1|0.3838383838383838|
|       0|0.6161616161616161|
+--------+------------------+



**Can you get the number of males and females?**


In [14]:
df_train.groupBy('Sex').count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



**1. What is the average number of survivors of each gender?**

In [15]:
df_train.groupBy('Sex').agg(avg('Survived')).show()#groupBy('Sex','Survived').count().select('Sex','Survived',col('count')/df_train.count()).show()

+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+





**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column.)

In [16]:
df_train.groupBy('Sex','Survived').count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



**Create temporary view PySpark:**

In [17]:
df_train.createOrReplaceTempView('titanic_train_data')

**How many people survived, and how many didn't survive? By SQL:**

In [18]:
spark.sql("""select Survived,COUNT(Survived)
            from titanic_train_data
            GROUP BY Survived""").show()

+--------+---------------+
|Survived|count(Survived)|
+--------+---------------+
|       1|            342|
|       0|            549|
+--------+---------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column.)

**Can you do this via SQL?**

In [19]:
spark.sql("""select Sex,COUNT(Survived) as Survived
            from titanic_train_data
            GROUP BY Sex""").show()

+------+--------+
|   Sex|Survived|
+------+--------+
|female|     314|
|  male|     577|
+------+--------+



**Display a ratio for p-class:**


In [20]:
spark.sql("""select Pclass,round(sum(Survived)/COUNT(Pclass),2 ) as Pclass_ratio,sum(Survived)
            from titanic_train_data
            GROUP BY Pclass""").show()
#Stuck Here

+------+------------+-------------+
|Pclass|Pclass_ratio|sum(Survived)|
+------+------------+-------------+
|     1|        0.63|          136|
|     3|        0.24|          119|
|     2|        0.47|           87|
+------+------------+-------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [21]:
df_all = df_train.union(df_test)

**Display count:**

In [22]:
df_all.count()

1329

**Temporary view PySpark:**

In [23]:
df_all.createOrReplaceTempView('all_data')

**Can you define the number of null values in each column?**


**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [24]:
df_all.select([count(when(col(c).isNull() , c)).alias(c) for c in df_all.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Preprocessing 

**Can you show me the name column from your temporary table?**

In [25]:
spark.sql("""select Name from all_data""").show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



**Run this code:**

In [26]:
from pyspark.sql import functions as F
combined = df_all.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display the title and count "Title" column:**

In [27]:
spark.sql("""select title,count(title) from combined group by title""").show()

+--------+------------+
|   title|count(title)|
+--------+------------+
|     Don|           1|
|    Miss|         257|
|Countess|           2|
|     Col|           4|
|     Rev|           9|
|    Lady|           2|
|  Master|          56|
|     Mme|           1|
|    Capt|           2|
|      Mr|         786|
|      Dr|          11|
|     Mrs|         186|
|     Sir|           2|
|Jonkheer|           2|
|    Mlle|           4|
|   Major|           3|
|      Ms|           1|
+--------+------------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [28]:
mapping = {'Miss':'Miss','Mrs':'Mrs','Mr':'Mr','Master':'Master','Ms':'Ms','Dr':'rare', 'Rev':'rare', 'Major':'rare', 'Col':'rare', 'Mlle':'rare', 'Capt':'rare', 'Don':'rare', 'Jonkheer':'rare', 'Countess':'rare', 'Ms':'rare', 'Sir':'rare', 'Lady':'rare','Mme':'rare'}
# df_rare = combined.replace(to_replace=mapping,subset=['Title'])
# df_rare.select('Title').show()

**Run the function:**

In [29]:
def impute_title(title):
    return mapping[title]

**Apply the function on "Title" column using UDF:**

In [30]:
from pyspark.sql.functions import udf
impute_title_udf = udf(lambda title:impute_title(title))
new_df = combined.withColumn('Title',impute_title_udf(col('Title')))

**Display "Title" from table and group by "Title" column:**

In [31]:
new_df.createOrReplaceTempView('combined')
spark.sql("""select title,count(title) from combined group by title""").show()

+------+------------+
| title|count(title)|
+------+------------+
|  rare|          44|
|  Miss|         257|
|Master|          56|
|    Mr|         786|
|   Mrs|         186|
+------+------------+



## **Preprocessing Age**

**Based on the age mean, you will fill in the missing age values:**

In [32]:
import numpy as np
age_avg = (combined.agg(avg('age')).collect())
new_df = new_df.na.fill(np.round(age_avg[0][0],0),subset=['Age'])

**Fill missing age with age mean:**

In [33]:
# i do it in on step

## **Preprocessing Embarked**

**Select Embarked, count them, order by count Desc, and save in grouped_Embarked variable:**




In [34]:
grouped_Embarked = spark.sql("""select Embarked , count(Embarked) as Embarked_Count from combined group by Embarked order by Embarked_Count desc """).show()

+--------+--------------+
|Embarked|Embarked_Count|
+--------+--------------+
|       S|           962|
|       C|           253|
|       Q|           111|
|    null|             0|
+--------+--------------+



**Show groupped_Embarked:**

**Get the groupped_Embarked:** 

**Fill missing values with Top 'S' of grouped_Embarked:**

In [35]:
new_df= new_df.na.fill('S',subset=['Embarked'])

## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [36]:
prep_df = new_df.withColumn('cabin',substring('cabin',0,1))

**Show the result:**

**Create the temporary view:**

In [37]:
prep_df.createOrReplaceTempView('prep_data')

**Select "Cabin" column, count Cabin column, Group by "Cabin" column, Order By count DESC**  

In [38]:
cabin_c=spark.sql("""select Cabin , count(Cabin) as cabin_count from prep_data group by Cabin order by cabin_count desc""")

**Fill missing values with "U":**

In [39]:
cabin_c.na.fill('U',subset=['Cabin'])

DataFrame[Cabin: string, cabin_count: bigint]

In [40]:
prep_df=prep_df.na.fill('U',subset=['Cabin'])

In [41]:
prep_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|    U|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|    C|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|    U|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|    C|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|    U|       S|    Mr|
|          6|       0|  

In [42]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

____________________________________________

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None) A feature transformer that merges multiple columns into a vector column.**



**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [43]:
combined = prep_df
X_train, X_test = combined.randomSplit([0.8,0.2],seed=0)
print(f"There are {X_train.count()} rows in the training set, and {X_test.count()} in the test set.")

There are 1060 rows in the training set, and 269 in the test set.


In [44]:
# X_train.dtypes

In [45]:
categoricalColumns = [col for (col, dtype) in X_train.dtypes
                   if dtype == "string"]
categoricalColumns           

['Name', 'Sex', 'Ticket', 'cabin', 'Embarked', 'Title']

In [46]:
categoricalColumns.remove("Name")
categoricalColumns

['Sex', 'Ticket', 'cabin', 'Embarked', 'Title']

In [47]:
# for col in categoricalColumns:
#   X_train.select(col).distinct().show()

In [48]:
indexOutputColumns = [x + "_Index" for x in categoricalColumns]
indexOutputColumns

['Sex_Index', 'Ticket_Index', 'cabin_Index', 'Embarked_Index', 'Title_Index']

In [49]:
oheOutputColumns = [x + "_OHE" for x in categoricalColumns]
oheOutputColumns

['Sex_OHE', 'Ticket_OHE', 'cabin_OHE', 'Embarked_OHE', 'Title_OHE']

In [50]:
stringIndexer = StringIndexer(inputCols=categoricalColumns,
                             outputCols=indexOutputColumns,
                             handleInvalid='skip')
oheEncoder = OneHotEncoder(inputCols=indexOutputColumns,
                          outputCols=oheOutputColumns)

In [51]:
numericColumns = [field for (field,dataType) in X_train.dtypes
              if ((dataType=='double'or dataType=='int')& (field!='Survived'))]
numericColumns.remove('PassengerId')
numericColumns

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [52]:
assemblerInputs = oheOutputColumns + numericColumns
assemblerInputs

['Sex_OHE',
 'Ticket_OHE',
 'cabin_OHE',
 'Embarked_OHE',
 'Title_OHE',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare']

In [57]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [58]:
rfc = RandomForestClassifier(featuresCol='features', labelCol='Survived', predictionCol='prediction', maxDepth=5)
pipeline =Pipeline(stages = [stringIndexer,oheEncoder,vecAssembler,rfc])

In [59]:
pipelineModel = pipeline.fit(X_train)

In [60]:
predDF = pipelineModel.transform(X_test)

In [61]:
predDF.select('features','Survived','prediction').show(5)

+--------------------+--------+----------+
|            features|Survived|prediction|
+--------------------+--------+----------+
|(638,[0,119,619,6...|       0|       0.0|
|(638,[381,619,628...|       1|       1.0|
|(638,[480,619,627...|       0|       0.0|
|(638,[309,621,627...|       1|       1.0|
|(638,[0,382,619,6...|       1|       0.0|
+--------------------+--------+----------+
only showing top 5 rows



**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [62]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
MCE = MulticlassClassificationEvaluator(predictionCol='prediction',
                                        labelCol='Survived',
                                        metricName='accuracy')

In [63]:
accuracy = MCE.evaluate(predDF)
accuracy

0.826530612244898